# A Repeat of The Analyses Done in Phase 1 Using Eveleigh

This presents a very similar analysis to the one done on our very simple office in the first phase of the project, mainly for interest of comparison.

All analyses prested here are conducted on a workplace with no HVAC and no air exchange between rooms.

## Imports

In [ ]:
import os
import pickle
from itertools import repeat
from multiprocessing import Pool
import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from apsrm import PCRTest, Person, Box
from apsrm.config import POOL_NCORES, DEFAULT_STRAIN
from apsrm.ext.multiprocessing import ProcessSeeder
from apsrm.ext.simulation import (
    create_pathogen,
    run_simulation,
    generate_means_tables,
    plot_histograms)
from utils import (
    create_workplace,
    create_emissions_calculator)

pathogen_name = DEFAULT_STRAIN

OUTPUT_BASE_DIR = '../../outputs/eveleigh'
OUTPUT_BASE_DIR = OUTPUT_BASE_DIR if os.path.exists(OUTPUT_BASE_DIR) else '.'
OUTPUT_PREFIX = pathogen_name
def opath(p): return os.path.join(OUTPUT_BASE_DIR, '{}_{}'.format(OUTPUT_PREFIX, p))

results_pickle = os.path.join(OUTPUT_BASE_DIR, 'all_results.pkl')
run_analysis = not os.path.exists(results_pickle)
process_seeder = ProcessSeeder()

def run_job(workplace, runner, R, intervention_name):
    if run_analysis:    
        process_seeder.reset()
        with Pool(POOL_NCORES, initializer=process_seeder) as pool:
            work = pool.imap_unordered(runner, range(R))
            return pd.DataFrame(tqdm(work, total=R)).assign(intervention=intervention_name), intervention_name

def no_reset_standard_runner(*args):
    return run_simulation(workplace, pathogen, emissions_calculator, pcrtest)

def standard_runner(*args):
    workplace.reset(full=True)
    return no_reset_standard_runner(*args)

def poc_runner(*args):
    workplace.reset(full=True)
    return run_simulation(workplace, pathogen, emissions_calculator, pcrtest, testing_fraction)

## Basic Parameters

In [ ]:
R = 10000
inter_box_acph = 1.

testing_fraction = 1.
mask_efficiency = .8
external_acph = 1.
hvac_acph = 7.
inter_box_acph = 1.
intervention_external_acph = 7.
hvac_return_filtering_efficiency = .85
air_cleaner_filtering_efficiency = .95,
air_cleaner_filtering_volume = 350.


pathogen = create_pathogen(pathogen_name)
emissions_calculator = create_emissions_calculator(pathogen)
pcrtest = PCRTest()

workplace = create_workplace(
    hvac_acph = hvac_acph,
    inter_box_acph = inter_box_acph,
    external_acph = external_acph)

## Simulations

### BAU

In [ ]:
results_bau = run_job(workplace, standard_runner, R, 'BAU')

### Efect of Masks

Masks are assumed to be 80% effective at preventing the wearer from becoming infected and reduces the infectivity of infected by the same amount.

In [ ]:
Person.ingestion_filter_efficiency = mask_efficiency
Person.shedding_filter_efficiency = mask_efficiency
results_masks = run_job(workplace, standard_runner, R, 'Masks Only')
Person.ingestion_filter_efficiency = 0.
Person.shedding_filter_efficiency = 0.

### Effect of Random Testing

Note the dramatic change in the time until detection. This has potentially large bearing on the risk in the broader community.

In [ ]:
results_testing = run_job(workplace, poc_runner, R, 'Testing Only')

### Effect of Random Testing and Masks

In [ ]:
Person.ingestion_filter_efficiency = mask_efficiency
Person.shedding_filter_efficiency = mask_efficiency
results_testing_and_masks = run_job(workplace, poc_runner, R, 'Masks and Testing')
Person.ingestion_filter_efficiency = 0.
Person.shedding_filter_efficiency = 0.

### Don't Allow Meetings

In [ ]:
workplace_generators = workplace._generators
workplace._generators = set()
results_no_meetings = run_job(workplace, standard_runner, R, 'No Meetings')
workplace._generators = workplace_generators

### Vaccinate Everyone in the Last Six Months

In [ ]:
from apsrm import Vaccine
vaccine = Vaccine()
def vaccinator(r):
    workplace.reset(full=True)
    for person, time in zip(workplace.persons, np.random.randint(-180, 0, size=len(workplace.persons))):
        person.vaccinate(time, vaccine)
    return no_reset_standard_runner(r)
results_vaccinate = run_job(workplace, vaccinator, R, 'Vaccinations')

### Increase Ventilation

Increase air exchange with the outside world.

In [ ]:
workplace = create_workplace(
    hvac_acph = hvac_acph,
    inter_box_acph = inter_box_acph,
    external_acph = intervention_external_acph)
results_ventilation = run_job(workplace, standard_runner, R, 'Increased Ventilation')

### HVAC Return Air Filters With 80% Efficiency 

In [ ]:
workplace = create_workplace(
    hvac_acph = hvac_acph,
    inter_box_acph = inter_box_acph,
    hvac_return_filtering_efficiency = hvac_return_filtering_efficiency)
results_hvac_return = run_job(workplace, standard_runner, R, 'HVAC Filters')

### Portable Air Filter With 95% Efficiency

Volume from https://pursuit.unimelb.edu.au/articles/which-air-cleaners-work-best-to-remove-aerosols-that-contain-viruses

In [ ]:
workplace = create_workplace(
    hvac_acph = hvac_acph,
    inter_box_acph = inter_box_acph,
    air_cleaner_filtering_efficiency = air_cleaner_filtering_efficiency,
    air_cleaner_filtering_volume = air_cleaner_filtering_volume)
results_portable_filters = run_job(workplace, standard_runner, R, 'Portable Filters')

In [ ]:
if run_analysis:        
    all_results = (
        results_bau,
        results_masks,
        results_testing,
        results_testing_and_masks,
        results_no_meetings,
        results_vaccinate,
        results_ventilation,
        results_hvac_return,
        results_portable_filters)
    
    with open(results_pickle, 'wb') as pkl:
        pickle.dump((all_results, R), pkl)
        
else:
    with open(results_pickle, 'rb') as pkl:
        all_results, R = pickle.load(pkl)

infection_counts = pd.concat([r[0] for r in all_results])
dt = CategoricalDtype(categories=[r[1] for r in all_results], ordered=True)
infection_counts['intervention'] = infection_counts['intervention'].astype(dt)

## Plots and Tables

In [ ]:
import matplotlib.font_manager as font_manager

# Add every font at the specified location
font_dir = ['../fnt']
for font in font_manager.findSystemFonts(font_dir):
    font_manager.fontManager.addfont(font)

# Set font family globally
plt.rcParams['font.family'] = 'Palatino'

In [ ]:
plot_histograms(infection_counts, OUTPUT_BASE_DIR, OUTPUT_PREFIX)

In [ ]:
means, means_latex = generate_means_tables(
    infection_counts, R,
    caption=r'Average number of workers infected and average first period in which a case is detected for each intervention considered in stage one, for the Data61 office in Eveleigh. The averages are taken over the simulations used to generate the histograms shown in Figures~\ref{fig:eveleigh:stage_1_comparison_delta_number_infected} and~\ref{fig:eveleigh:stage_1_comparisondelta_period_finished}.',
    label='tab:eveleigh:stage1_delta_means')

with open(opath('means.tex'), 'w') as outf: outf.write(means_latex)
print(means)